In [53]:
import pandas as pd
from docx import Document

# Leer el archivo CSV
df = pd.read_csv('RBE.csv', low_memory=False)

# Especificar columnas de interés
columns_of_interest = ['TEMP_MEDIA_RBE', 'CONF_TEMP_MEDIA_RBE', 'TEMP_MIN_RBE', 'CONF_TEMP_MIN_RBE', 'TEMP_MAX_RBE', 'CONF_TEMP_MAX_RBE', 
                       'PP_RBE', 'CONF_PP_RBE']

# Calcular los datos faltantes como el número de filas con al menos un NaN en las columnas por cada grupo
missing_data_counts = df.groupby('ID_CIDERH').apply(lambda x: x[columns_of_interest].isnull().sum().sum())

# Contar el total de datos por grupo
total_data_counts = df.groupby('ID_CIDERH').size()

# Convertir las fechas y calcular la cantidad de años de datos disponibles
df['FECHA_RBE'] = pd.to_datetime(df['FECHA_RBE'], errors='coerce')
years_of_data = df.groupby('ID_CIDERH').apply(lambda x: x['FECHA_RBE'].max().year - x['FECHA_RBE'].min().year)

# Crear el DataFrame de resumen
summary_df = pd.DataFrame({
    'ID EM': years_of_data.index,
    'Cantidad de años': years_of_data.values,
    'Cantidad de datos': total_data_counts.values,
    'Datos faltantes': missing_data_counts.values
})


summary_df.reset_index(drop=True, inplace=True)

# Crear word
doc = Document()
doc.add_heading('Análisis de calidad de los datos de las EM', 0)

# Agregar la tabla al documento
table = doc.add_table(rows=1, cols=4)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'ID EM'
hdr_cells[1].text = 'Cantidad de años'
hdr_cells[2].text = 'Cantidad de datos'
hdr_cells[3].text = 'Datos faltantes'

# Llenar la tabla con datos
for index, row in summary_df.iterrows():
    row_cells = table.add_row().cells
    row_cells[0].text = str(row['ID EM'])
    row_cells[1].text = str(row['Cantidad de años'])
    row_cells[2].text = str(row['Cantidad de datos'])
    row_cells[3].text = str(row['Datos faltantes'])

# Guardar el documento
doc.save('Resumen_Estaciones.docx')

In [3]:
import pandas as pd
from docx import Document
#ANALISIS DE PRECIPITACIÓN
# Cargar el archivo CSV
archivo_csv = 'RBE.csv'
datos = pd.read_csv(archivo_csv, low_memory=False)

# Realizar análisis exploratorio
resultados = datos.groupby('ID_CIDERH')['PP_RBE'].describe()

# Calcular IQR
resultados['IQR'] = resultados['75%'] - resultados['25%']

# Renombrar columnas para mayor claridad
resultados.rename(columns={'mean': 'Media', 
                           'std': 'D. Estándar',
                           'min': 'Min',
                           '25%': 'Q1',
                           '50%': 'Mediana',
                           '75%': 'Q3',
                           'max': 'Max'}, inplace=True)

# Formatear columnas específicas con notación científica para valores muy pequeños y manejo de casos especiales
def custom_format(val, normal_format, sci_threshold=1e-4):
    special_cases = {0: "0", -99.9: "-99.9", 99.9: "99.9"}
    if val in special_cases:
        return special_cases[val]
    elif abs(val) < sci_threshold:
        return "{:.2e}".format(val)
    else:
        return normal_format.format(val)

format_specs = {
    'Media': "{:.3f}",
    'D. Estándar': "{:.8f}",
    'Q1': "{:.1f}",
    'Q3': "{:.1f}",
    'IQR': "{:.2f}",
    'Min': "{:.1f}",
    'Max': "{:.1f}",
    'Mediana': "{:.1f}"
}

for column, format_spec in format_specs.items():
    resultados[column] = resultados[column].apply(lambda x: custom_format(x, format_spec))

# Quitar la columna 'count' de los resultados
resultados = resultados.drop(columns=['count'])

resultados.reset_index(inplace=True)

# Crear un nuevo documento de Word
doc = Document()

# Añadir un título al documento
doc.add_heading('Resultados del Análisis de PP_RBE', 0)

# Añadir la tabla al documento
tabla = doc.add_table(rows=1, cols=len(resultados.columns))

# Añadir los encabezados de la tabla
hdr_cells = tabla.rows[0].cells
for i, col_name in enumerate(resultados.columns):
    hdr_cells[i].text = str(col_name)

# Añadir las filas de la tabla
for index, row in resultados.iterrows():
    row_cells = tabla.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Guardar el documento
doc.save('analisis_PP_RBE.docx')

In [4]:
import pandas as pd
from docx import Document
#ANALISIS DE TEMP. MINIMA
# Cargar el archivo CSV
archivo_csv = 'RBE.csv'
datos = pd.read_csv(archivo_csv, low_memory=False)

# Realizar análisis exploratorio
resultados = datos.groupby('ID_CIDERH')['TEMP_MIN_RBE'].describe()

# Calcular IQR
resultados['IQR'] = resultados['75%'] - resultados['25%']

# Renombrar columnas para mayor claridad
resultados.rename(columns={'mean': 'Media', 
                           'std': 'D. Estándar',
                           'min': 'Min',
                           '25%': 'Q1',
                           '50%': 'Mediana',
                           '75%': 'Q3',
                           'max': 'Max'}, inplace=True)

# Formatear columnas específicas con notación científica para valores muy pequeños y manejo de casos especiales
def custom_format(val, normal_format, sci_threshold=1e-4):
    special_cases = {0: "0", -99.9: "-99.9", 99.9: "99.9"}
    if val in special_cases:
        return special_cases[val]
    elif abs(val) < sci_threshold:
        return "{:.2e}".format(val)
    else:
        return normal_format.format(val)

format_specs = {
    'Media': "{:.3f}",
    'D. Estándar': "{:.8f}",
    'Q1': "{:.1f}",
    'Q3': "{:.1f}",
    'IQR': "{:.2f}",
    'Min': "{:.1f}",
    'Max': "{:.1f}",
    'Mediana': "{:.1f}"
}

for column, format_spec in format_specs.items():
    resultados[column] = resultados[column].apply(lambda x: custom_format(x, format_spec))

# Quitar la columna 'count' de los resultados
resultados = resultados.drop(columns=['count'])

resultados.reset_index(inplace=True)

# Crear un nuevo documento de Word
doc = Document()

# Añadir un título al documento
doc.add_heading('Resultados del Análisis de TEMP_MIN_RBE', 0)

# Añadir la tabla al documento
tabla = doc.add_table(rows=1, cols=len(resultados.columns))

# Añadir los encabezados de la tabla
hdr_cells = tabla.rows[0].cells
for i, col_name in enumerate(resultados.columns):
    hdr_cells[i].text = str(col_name)

# Añadir las filas de la tabla
for index, row in resultados.iterrows():
    row_cells = tabla.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Guardar el documento
doc.save('analisis_TEMP_MIN_RBE.docx')

In [5]:
import pandas as pd
from docx import Document
#ANALISIS DE TEMP. MEDIA
# Cargar el archivo CSV
archivo_csv = 'RBE.csv'
datos = pd.read_csv(archivo_csv, low_memory=False)

# Realizar análisis exploratorio
resultados = datos.groupby('ID_CIDERH')['TEMP_MEDIA_RBE'].describe()

# Calcular IQR
resultados['IQR'] = resultados['75%'] - resultados['25%']

# Renombrar columnas para mayor claridad
resultados.rename(columns={'mean': 'Media', 
                           'std': 'D. Estándar',
                           'min': 'Min',
                           '25%': 'Q1',
                           '50%': 'Mediana',
                           '75%': 'Q3',
                           'max': 'Max'}, inplace=True)

# Formatear columnas específicas con notación científica para valores muy pequeños y manejo de casos especiales
def custom_format(val, normal_format, sci_threshold=1e-4):
    special_cases = {0: "0", -99.9: "-99.9", 99.9: "99.9"}
    if val in special_cases:
        return special_cases[val]
    elif abs(val) < sci_threshold:
        return "{:.2e}".format(val)
    else:
        return normal_format.format(val)

format_specs = {
    'Media': "{:.3f}",
    'D. Estándar': "{:.8f}",
    'Q1': "{:.1f}",
    'Q3': "{:.1f}",
    'IQR': "{:.2f}",
    'Min': "{:.1f}",
    'Max': "{:.1f}",
    'Mediana': "{:.1f}"
}

for column, format_spec in format_specs.items():
    resultados[column] = resultados[column].apply(lambda x: custom_format(x, format_spec))

# Quitar la columna 'count' de los resultados
resultados = resultados.drop(columns=['count'])

resultados.reset_index(inplace=True)

# Crear un nuevo documento de Word
doc = Document()

# Añadir un título al documento
doc.add_heading('Resultados del Análisis de TEMP_MEDIA_RBE', 0)

# Añadir la tabla al documento
tabla = doc.add_table(rows=1, cols=len(resultados.columns))

# Añadir los encabezados de la tabla
hdr_cells = tabla.rows[0].cells
for i, col_name in enumerate(resultados.columns):
    hdr_cells[i].text = str(col_name)

# Añadir las filas de la tabla
for index, row in resultados.iterrows():
    row_cells = tabla.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Guardar el documento
doc.save('analisis_TEMP_MEDIA_RBE.docx')

In [6]:
import pandas as pd
from docx import Document
#ANALISIS DE TEMP. MAXIMA
# Cargar el archivo CSV
archivo_csv = 'RBE.csv'
datos = pd.read_csv(archivo_csv, low_memory=False)

# Realizar análisis exploratorio
resultados = datos.groupby('ID_CIDERH')['TEMP_MAX_RBE'].describe()

# Calcular IQR
resultados['IQR'] = resultados['75%'] - resultados['25%']

# Renombrar columnas para mayor claridad
resultados.rename(columns={'mean': 'Media', 
                           'std': 'D. Estándar',
                           'min': 'Min',
                           '25%': 'Q1',
                           '50%': 'Mediana',
                           '75%': 'Q3',
                           'max': 'Max'}, inplace=True)

# Formatear columnas específicas con notación científica para valores muy pequeños y manejo de casos especiales
def custom_format(val, normal_format, sci_threshold=1e-4):
    special_cases = {0: "0", -99.9: "-99.9", 99.9: "99.9"}
    if val in special_cases:
        return special_cases[val]
    elif abs(val) < sci_threshold:
        return "{:.2e}".format(val)
    else:
        return normal_format.format(val)

format_specs = {
    'Media': "{:.3f}",
    'D. Estándar': "{:.8f}",
    'Q1': "{:.1f}",
    'Q3': "{:.1f}",
    'IQR': "{:.2f}",
    'Min': "{:.1f}",
    'Max': "{:.1f}",
    'Mediana': "{:.1f}"
}

for column, format_spec in format_specs.items():
    resultados[column] = resultados[column].apply(lambda x: custom_format(x, format_spec))

# Quitar la columna 'count' de los resultados
resultados = resultados.drop(columns=['count'])

resultados.reset_index(inplace=True)

# Crear un nuevo documento de Word
doc = Document()

# Añadir un título al documento
doc.add_heading('Resultados del Análisis de TEMP_MAX_RBE', 0)

# Añadir la tabla al documento
tabla = doc.add_table(rows=1, cols=len(resultados.columns))

# Añadir los encabezados de la tabla
hdr_cells = tabla.rows[0].cells
for i, col_name in enumerate(resultados.columns):
    hdr_cells[i].text = str(col_name)

# Añadir las filas de la tabla
for index, row in resultados.iterrows():
    row_cells = tabla.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Guardar el documento
doc.save('analisis_TEMP_MAX_RBE.docx')